In [2]:
import os, warnings
warnings.filterwarnings(action='ignore')

In [3]:
from IPython.display import display, Image

In [4]:
import tensorflow as tf
print(tf.__version__)

1.15.0


In [6]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/",one_hot=True)
type(mnist)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

tensorflow.contrib.learn.python.learn.datasets.base.Datasets

In [11]:
print(mnist.train.labels.shape)
print(mnist.train.images.shape)

(55000, 10)
(55000, 784)


### Placeholder 지정
* X 입력데이터
* Y 출력데이터


In [12]:
X = tf.placeholder(tf.float32,[None,784])
Y = tf.placeholder(tf.float32,[None,10])

### 미니배치
* 이미지를 하나씩 학습시키는것 보다 한번에 학습시키는것
* [None,784] None은 한번에 학습시킨 이미지의 개수를 지정하는 값이 들어감

### 모델(신경망 만들기)
* 입력층 - 784(28*28) ->
    * 256 (첫번째 은닉층 뉴런) -> 
    * 256 (두번째 은닉층 뉴런) ->
* 출력층 - 10 (결과값 0~9), 범주형의 종류의 개수

In [14]:
keep_prob = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([784,256],stddev=0.01))
L1 = tf.nn.relu(tf.matmul(X,W1))
L1 = tf.nn.dropout(L1,keep_prob)

W2 = tf.Variable(tf.random_normal([256,256],stddev=0.01))
L2 = tf.nn.relu(tf.matmul(L1,W2))
L2 = tf.nn.dropout(L2,keep_prob)

W3 = tf.Variable(tf.random_normal([256,10],stddev=0.01))
model = tf.matmul(L2,W3)

print(W3)
print(model)

<tf.Variable 'Variable_5:0' shape=(256, 10) dtype=float32_ref>
Tensor("MatMul_5:0", shape=(?, 10), dtype=float32)


In [15]:
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model,labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [16]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

학습
* 사전준비 : 배치사이즈(batch_size) 한번 학습시킬 데이터수
* 전체 데이터/배치사이즈 : 몇번 학습을 하는가(total_batch)

In [19]:
batch_size = 100
total_batch = int(mnist.train.num_examples/batch_size)

In [25]:
for epoch in range(10):
    total_cost = 0
    for i in range(total_batch):
        batch_xs,batch_ys = mnist.train.next_batch(batch_size)
        _, cost_val = sess.run([optimizer,cost],feed_dict={X:batch_xs,Y:batch_ys,
                                                          keep_prob:0.8})
        total_cost = total_cost + cost_val
    print("Epoch : %4d" % (epoch + 1),
         '평균 Cost = ', "{:.3f}".format(total_cost/total_batch),
         epoch,'의 합계 ', total_cost)
    
print("최적화 완료")

Epoch :    1 평균 Cost =  0.014 0 의 합계  7.488322103712562
Epoch :    2 평균 Cost =  0.016 1 의 합계  8.623157990105028
Epoch :    3 평균 Cost =  0.017 2 의 합계  9.256238108730031
Epoch :    4 평균 Cost =  0.014 3 의 합계  7.870813639347034
Epoch :    5 평균 Cost =  0.016 4 의 합계  8.66401692143154
Epoch :    6 평균 Cost =  0.013 5 의 합계  7.401299494302293
Epoch :    7 평균 Cost =  0.015 6 의 합계  8.309689684670957
Epoch :    8 평균 Cost =  0.015 7 의 합계  8.166893303343386
Epoch :    9 평균 Cost =  0.013 8 의 합계  7.168700589279069
Epoch :   10 평균 Cost =  0.015 9 의 합계  8.03414007779429
최적화 완료


정확도 확인
* argmax : 가장 큰 값의 갖는 인덱스를 반환

In [27]:
# is_correct = tf.equal(A,B) # A,B가 같으면 True, 아니면 False
is_correct = tf.equal(tf.argmax(model,1),tf.argmax(Y,1)) # 같은가?
accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))
print("정확도 : " ,sess.run(accuracy,feed_dict = {X:mnist.test.images,
                                              Y:mnist.test.labels,
                                             keep_prob: 0.8}))

정확도 :  0.9772
